# Data Exploration

This notebook provides an initial exploration of the financial data to understand:
- Data structure and quality
- Basic statistics and distributions
- Missing values and data types
- Initial insights and patterns

## Setup and Imports

In [ ]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

# Import project modules
from src.data_processing.database_manager import DatabaseManager
from src.data_processing.excel_importer import ExcelImporter
from src.data_processing.data_cleaner import DataCleaner

print("Setup complete!")

## Load Data

In [ ]:
# Initialize database manager
db_manager = DatabaseManager()

# Load financial data
query = """
SELECT date, amount, category, subcategory, description, 
       store_location, payment_method, created_at
FROM financial_data
ORDER BY date DESC
"""

df = db_manager.query_data(query)
print(f"Loaded {len(df):,} records")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")

# Convert date column
df['date'] = pd.to_datetime(df['date'])

df.head()

## Data Overview

In [ ]:
# Basic information about the dataset
print("Dataset Shape:", df.shape)
print("\nColumn Information:")
print(df.info())

print("\nData Types:")
print(df.dtypes)

print("\nMemory Usage:")
print(f"{df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
# Missing values analysis
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing_values,
    'Missing Percentage': missing_percentage
})

missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)

print("Missing Values Summary:")
print(missing_df)

# Visualize missing values
if not missing_df.empty:
    fig, ax = plt.subplots(figsize=(10, 6))
    missing_df['Missing Percentage'].plot(kind='bar', ax=ax)
    ax.set_title('Missing Values by Column')
    ax.set_ylabel('Percentage Missing')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("No missing values found!")

## Basic Statistics

In [ ]:
# Descriptive statistics for numerical columns
print("Numerical Statistics:")
print(df.describe())

# Amount statistics
print("\nAmount Statistics:")
print(f"Total Amount: ${df['amount'].sum():,.2f}")
print(f"Mean Amount: ${df['amount'].mean():,.2f}")
print(f"Median Amount: ${df['amount'].median():,.2f}")
print(f"Standard Deviation: ${df['amount'].std():,.2f}")
print(f"Min Amount: ${df['amount'].min():,.2f}")
print(f"Max Amount: ${df['amount'].max():,.2f}")

In [ ]:
# Categorical data analysis
categorical_columns = ['category', 'subcategory', 'store_location', 'payment_method']

for col in categorical_columns:
    if col in df.columns and not df[col].isnull().all():
        print(f"\n{col.replace('_', ' ').title()} Distribution:")
        value_counts = df[col].value_counts().head(10)
        print(value_counts)
        print(f"Unique values: {df[col].nunique()}")

## Data Visualizations

In [ ]:
# Amount distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Histogram
axes[0, 0].hist(df['amount'], bins=50, alpha=0.7, edgecolor='black')
axes[0, 0].set_title('Amount Distribution')
axes[0, 0].set_xlabel('Amount')
axes[0, 0].set_ylabel('Frequency')

# Box plot
axes[0, 1].boxplot(df['amount'])
axes[0, 1].set_title('Amount Box Plot')
axes[0, 1].set_ylabel('Amount')

# Log scale histogram (if positive values)
positive_amounts = df[df['amount'] > 0]['amount']
if len(positive_amounts) > 0:
    axes[1, 0].hist(np.log10(positive_amounts), bins=50, alpha=0.7, edgecolor='black')
    axes[1, 0].set_title('Amount Distribution (Log Scale)')
    axes[1, 0].set_xlabel('Log10(Amount)')
    axes[1, 0].set_ylabel('Frequency')

# Cumulative distribution
sorted_amounts = np.sort(df['amount'])
y = np.arange(1, len(sorted_amounts) + 1) / len(sorted_amounts)
axes[1, 1].plot(sorted_amounts, y)
axes[1, 1].set_title('Cumulative Distribution')
axes[1, 1].set_xlabel('Amount')
axes[1, 1].set_ylabel('Cumulative Probability')

plt.tight_layout()
plt.show()

In [ ]:
# Time series analysis
# Daily aggregation
daily_amounts = df.groupby(df['date'].dt.date)['amount'].agg(['sum', 'count', 'mean']).reset_index()
daily_amounts['date'] = pd.to_datetime(daily_amounts['date'])

fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# Daily total amounts
axes[0].plot(daily_amounts['date'], daily_amounts['sum'])
axes[0].set_title('Daily Total Amount')
axes[0].set_ylabel('Total Amount')
axes[0].grid(True)

# Daily transaction count
axes[1].plot(daily_amounts['date'], daily_amounts['count'], color='orange')
axes[1].set_title('Daily Transaction Count')
axes[1].set_ylabel('Number of Transactions')
axes[1].grid(True)

# Daily average amount
axes[2].plot(daily_amounts['date'], daily_amounts['mean'], color='green')
axes[2].set_title('Daily Average Transaction Amount')
axes[2].set_xlabel('Date')
axes[2].set_ylabel('Average Amount')
axes[2].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Category analysis
if 'category' in df.columns and not df['category'].isnull().all():
    category_stats = df.groupby('category')['amount'].agg(['sum', 'count', 'mean']).sort_values('sum', ascending=False)
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Total amount by category
    top_categories = category_stats.head(10)
    axes[0, 0].bar(range(len(top_categories)), top_categories['sum'])
    axes[0, 0].set_title('Total Amount by Category (Top 10)')
    axes[0, 0].set_ylabel('Total Amount')
    axes[0, 0].set_xticks(range(len(top_categories)))
    axes[0, 0].set_xticklabels(top_categories.index, rotation=45, ha='right')
    
    # Transaction count by category
    axes[0, 1].bar(range(len(top_categories)), top_categories['count'], color='orange')
    axes[0, 1].set_title('Transaction Count by Category (Top 10)')
    axes[0, 1].set_ylabel('Number of Transactions')
    axes[0, 1].set_xticks(range(len(top_categories)))
    axes[0, 1].set_xticklabels(top_categories.index, rotation=45, ha='right')
    
    # Average amount by category
    avg_sorted = category_stats.sort_values('mean', ascending=False).head(10)
    axes[1, 0].bar(range(len(avg_sorted)), avg_sorted['mean'], color='green')
    axes[1, 0].set_title('Average Amount by Category (Top 10)')
    axes[1, 0].set_ylabel('Average Amount')
    axes[1, 0].set_xticks(range(len(avg_sorted)))
    axes[1, 0].set_xticklabels(avg_sorted.index, rotation=45, ha='right')
    
    # Pie chart for top categories
    top_5_categories = category_stats.head(5)
    axes[1, 1].pie(top_5_categories['sum'], labels=top_5_categories.index, autopct='%1.1f%%')
    axes[1, 1].set_title('Amount Distribution (Top 5 Categories)')
    
    plt.tight_layout()
    plt.show()
    
    print("\nCategory Statistics:")
    print(category_stats.head(10))

## Data Quality Assessment

In [ ]:
# Use data cleaner to assess quality
cleaner = DataCleaner()
quality_report = cleaner.validate_data_quality(df)

print("Data Quality Report:")
print(f"Quality Score: {quality_report['quality_score']:.2f}/100")
print(f"Total Rows: {quality_report['total_rows']:,}")
print(f"Total Columns: {quality_report['total_columns']}")
print(f"Duplicate Rows: {quality_report['duplicate_rows']:,}")
print(f"Memory Usage: {quality_report['memory_usage'] / 1024**2:.2f} MB")

if quality_report['missing_required_columns']:
    print(f"Missing Required Columns: {quality_report['missing_required_columns']}")
else:
    print("All required columns present")

print("\nMissing Values by Column:")
for col, missing_count in quality_report['missing_values'].items():
    if missing_count > 0:
        percentage = (missing_count / quality_report['total_rows']) * 100
        print(f"  {col}: {missing_count:,} ({percentage:.1f}%)")

## Outlier Detection

In [ ]:
# Detect outliers using IQR method
Q1 = df['amount'].quantile(0.25)
Q3 = df['amount'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['amount'] < lower_bound) | (df['amount'] > upper_bound)]

print(f"Outlier Detection (IQR Method):")
print(f"Lower Bound: ${lower_bound:.2f}")
print(f"Upper Bound: ${upper_bound:.2f}")
print(f"Number of Outliers: {len(outliers):,} ({len(outliers)/len(df)*100:.2f}%)")

if len(outliers) > 0:
    print(f"\nOutlier Statistics:")
    print(f"Min Outlier: ${outliers['amount'].min():.2f}")
    print(f"Max Outlier: ${outliers['amount'].max():.2f}")
    print(f"Mean Outlier: ${outliers['amount'].mean():.2f}")
    
    # Show top 10 largest outliers
    print("\nTop 10 Largest Outliers:")
    top_outliers = outliers.nlargest(10, 'amount')[['date', 'amount', 'category', 'description']]
    print(top_outliers)

## Correlation Analysis

In [ ]:
# Create numerical features for correlation analysis
df_corr = df.copy()

# Add time-based features
df_corr['year'] = df_corr['date'].dt.year
df_corr['month'] = df_corr['date'].dt.month
df_corr['day_of_week'] = df_corr['date'].dt.dayofweek
df_corr['day_of_month'] = df_corr['date'].dt.day
df_corr['hour'] = df_corr['date'].dt.hour

# Select numerical columns
numerical_cols = df_corr.select_dtypes(include=[np.number]).columns
correlation_matrix = df_corr[numerical_cols].corr()

# Plot correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5)
plt.title('Correlation Matrix')
plt.tight_layout()
plt.show()

# Show correlations with amount
if 'amount' in correlation_matrix.columns:
    amount_correlations = correlation_matrix['amount'].abs().sort_values(ascending=False)
    print("\nCorrelations with Amount:")
    print(amount_correlations)

## Summary and Next Steps

In [ ]:
print("DATA EXPLORATION SUMMARY")
print("=" * 50)

print(f"Dataset Size: {len(df):,} rows, {len(df.columns)} columns")
print(f"Date Range: {df['date'].min().date()} to {df['date'].max().date()}")
print(f"Total Amount: ${df['amount'].sum():,.2f}")
print(f"Average Transaction: ${df['amount'].mean():.2f}")
print(f"Data Quality Score: {quality_report['quality_score']:.1f}/100")

if 'category' in df.columns:
    print(f"Number of Categories: {df['category'].nunique()}")
    print(f"Top Category: {df.groupby('category')['amount'].sum().idxmax()}")

print(f"Outliers Detected: {len(outliers):,} ({len(outliers)/len(df)*100:.1f}%)")

missing_cols = [col for col, count in quality_report['missing_values'].items() if count > 0]
if missing_cols:
    print(f"Columns with Missing Data: {', '.join(missing_cols)}")
else:
    print("No missing data detected")

print("\nNEXT STEPS:")
print("1. Review and clean any data quality issues")
print("2. Investigate outliers for validity")
print("3. Perform detailed financial analysis")
print("4. Analyze trends and patterns over time")
print("5. Build predictive models if needed")